# Data processing with Pandas 2

This week we will continue developing our skills using [Pandas](https://pandas.pydata.org/) to process real data. 

## Motivation

![Finland April 2019](img/Finland-April-2019.png)
*Source: [https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa](https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa)*

April 2019 was the [second warmest April on record globally](https://weather.com/news/climate/news/2019-05-20-april-2019-global-temperatures-nasa-noaa), and the warmest on record at 13 weather stations in Finland. 
In this lesson, we will use our data manipulation and analysis skills to analyze weather data from Finland, and investigate the claim that April 2019 was the warmest on record across Finland.

Along the way we will cover a number of useful techniques in pandas including:

- renaming columns
- iterating data frame rows and applying functions
- data aggregation
- repeating the analysis task for several input files

## Input data

In the lesson this week we are using weather observation data from Finland [downloaded from NOAA](https://www7.ncdc.noaa.gov/CDO/cdopoemain.cmd?datasetabbv=DS3505&countryabbv=&georegionabbv=&resolution=40). You will be working with data from either 15 or 4 different weather observation stations from Finland, depending on your environment.





## About the data

As part of the download there are a number of files that describe the weather data. These *metadata* files include:

- A list of stations\*: [data/6367598020644stn.txt](metadata/6367598020644stn.txt)
- Details about weather observations at each station: [data/6367598020644inv.txt](metadata/6367598020644inv.txt)
- A data description (i.e., column names): [data/3505doc.txt](metadata/3505doc.txt)

\*Note that the list of stations is for all 15 stations, even if you're working with only the 4 stations on the CSC Notebooks platform.

The input data for this week are separated with varying number of spaces (i.e., fixed width). The first lines and columns of the data look like following:

``` 
  USAF  WBAN YR--MODAHRMN DIR SPD GUS CLG SKC L M H  VSB MW MW MW MW AW AW AW AW W TEMP DEWP    SLP   ALT    STP MAX MIN PCP01 PCP06 PCP24 PCPXX SD
029440 99999 190601010600 090   7 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1011.0 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601011300 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   27 **** 1015.5 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601012000 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *   25 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** ** 
029440 99999 190601020600 ***   0 *** *** CLR * * *  0.0 ** ** ** ** ** ** ** ** *   26 **** 1016.2 ***** ****** *** *** ***** ***** ***** ***** **
```

We will develop our analysis workflow using data for a single station. Then, we will repeat the same process for all the stations.

## Reading the data

In order to get started, let's import pandas: 

In [4]:
!pip install pandas
import pandas as pd

     |████████████████████████████████| 9.9 MB 15.3 MB/s eta 0:00:01
     |████████████████████████████████| 15.3 MB 61.2 MB/s eta 0:00:01


At this point, you can already have a quick look at the input file `029440.txt` for Tampere Pirkkala and how it is structured. We can notice at least two things we need to consider when reading in the data:

<div class="alert alert-info">

**Input data structure**
    
- **Delimiter:** The data are **separated with a varying amount of spaces**. If you check out the documentation for the [read_csv() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), you can see that there are two different ways of doing this. We can use either `sep='\s+'` or `delim_whitespace=True` (but not both at the same time). In this case, we prefer to use `delim_whitespace` parameter.

- **No Data values:** No data values in the NOAA data are coded with varying number of `*`. We can tell pandas to consider those characters as NaNs by specifying `na_values=['*', '**', '***', '****', '*****', '******']`.
</div>

In [5]:
# Define relative path to the file
fp = r'data/029070.txt'

# Read data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, na_values=['*', '**', '***', '****', '*****', '******'])

/srv/conda/envs/notebook/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Let's see how the data looks by printing the first five rows with the `head()` function:

In [6]:
data.head()


USAF   WBAN  YR--MODAHRMN    DIR   SPD  GUS  CLG  SKC   L   M  ...  \
0  29070  99999  190101010600  270.0  36.0  NaN  NaN  OVC NaN NaN  ...   
1  29070  99999  190101011300  290.0  18.0  NaN  NaN  SCT NaN NaN  ...   
2  29070  99999  190101012000    NaN   0.0  NaN  NaN  OVC NaN NaN  ...   
3  29070  99999  190101020600  180.0  18.0  NaN  NaN  OVC NaN NaN  ...   
4  29070  99999  190101021300  180.0  22.0  NaN  NaN  OVC NaN NaN  ...   

      SLP  ALT  STP  MAX  MIN  PCP01  PCP06  PCP24  PCPXX  SD  
0  1020.0  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
1  1020.0  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
2  1020.0  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
3  1018.3  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
4  1017.6  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  

[5 rows x 33 columns]

All seems ok. However, we won't be needing all of the 33 columns for detecting warm temperatures in April. We can check all column names by running `data.columns`:

In [7]:
data.columns

Index(['USAF', 'WBAN', 'YR--MODAHRMN', 'DIR', 'SPD', 'GUS', 'CLG', 'SKC', 'L',
       'M', 'H', 'VSB', 'MW', 'MW.1', 'MW.2', 'MW.3', 'AW', 'AW.1', 'AW.2',
       'AW.3', 'W', 'TEMP', 'DEWP', 'SLP', 'ALT', 'STP', 'MAX', 'MIN', 'PCP01',
       'PCP06', 'PCP24', 'PCPXX', 'SD'],
      dtype='object')

A description for all these columns is available in the metadata file [data/3505doc.txt](metadata/3505doc.txt). 

**Let's read in the data one more time.** This time, we will read in only some of the columns using the `usecols` parameter. Let's read in columns that might be somehow useful to our analysis, or at least that contain some values that are meaningful to us, including the station name, timestamp, and data about wind and temperature: `'USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'`

In [8]:
# Read in only selected columns
data = pd.read_csv(fp, delim_whitespace=True, 
                   usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], 
                   na_values=['*', '**', '***', '****', '*****', '******'])

# Check the dataframe
data.head()

USAF  YR--MODAHRMN    DIR   SPD  GUS  TEMP  MAX  MIN
0  29070  190101010600  270.0  36.0  NaN  18.0  NaN  NaN
1  29070  190101011300  290.0  18.0  NaN  19.0  NaN  NaN
2  29070  190101012000    NaN   0.0  NaN  15.0  NaN  NaN
3  29070  190101020600  180.0  18.0  NaN  21.0  NaN  NaN
4  29070  190101021300  180.0  22.0  NaN  22.0  NaN  NaN

Okay so we can see that the data was successfully read to the DataFrame and we also seemed to be able to convert the asterisk (\*) characters into `NaN` values. 

## Renaming columns

As we saw above some of the column names are a bit awkward and difficult to interpret. Luckily, it is easy to alter labels in a pandas DataFrame using the [rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)-function. In order to change the column names, we need to tell pandas how we want to rename the columns using a dictionary that lists old and new column names

Let's first check again the current column names in our DataFrame:

In [9]:
data.columns

Index(['USAF', 'YR--MODAHRMN', 'DIR', 'SPD', 'GUS', 'TEMP', 'MAX', 'MIN'], dtype='object')

<div class="alert alert-info">

**Dictionaries**

A [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) is a specific data structure in Python for storing key-value pairs. During this course, we will use dictionaries mainly when renaming columns in a pandas series, but dictionaries are useful for many different purposes! For more information about Python dictionaries, check out [this tutorial](https://realpython.com/python-dicts/).
</div>

We can define the new column names using a [dictionary](https://www.tutorialspoint.com/python/python_dictionary.htm) where we list "`key: value`" pairs, in which the original column name (the one which will be replaced) is the key and the new column name is the value.

- Let's change the following:
   
   - `YR--MODAHRMN` to `TIME`
   - `SPD` to `SPEED`
   - `GUS` to `GUST`

In [10]:
# Create the dictionary with old and new names
new_names = {'YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST'}

# Let's see what the variable new_names look like
new_names

{'YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST'}

In [11]:
# Check the data type of the new_names variable
type(new_names)

dict

From above we can see that we have successfully created a new dictionary. 

Now we can change the column names by passing that dictionary using the parameter `columns` in the `rename()` function:

In [12]:
# Rename the columns
data = data.rename(columns=new_names)

# Print the new columns
print(data.columns)

Index(['USAF', 'TIME', 'DIR', 'SPEED', 'GUST', 'TEMP', 'MAX', 'MIN'], dtype='object')


Perfect, now our column names are easier to understand and use. 

#### Check your understanding

The temperature values in our data files are again in Fahrenheit. As you might guess, we will soon convert these temperatures in to Celsius. In order to avoid confusion with the columns, let's rename the column `TEMP` to `TEMP_F`. Let's also rename `USAF` to `STATION_NUMBER`.

In [13]:
# Solution
# Create the dictionary with old and new names
new_names = {'USAF':'STATION_NUMBER', 'TEMP': 'TEMP_F'}

# Rename the columns
data = data.rename(columns=new_names)

# Check the output
data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN
0           29070  190101010600  270.0   36.0   NaN    18.0  NaN  NaN
1           29070  190101011300  290.0   18.0   NaN    19.0  NaN  NaN
2           29070  190101012000    NaN    0.0   NaN    15.0  NaN  NaN
3           29070  190101020600  180.0   18.0   NaN    21.0  NaN  NaN
4           29070  190101021300  180.0   22.0   NaN    22.0  NaN  NaN

## Data properties

As we learned last week, it's always a good idea to check basic properties of the input data before proceeding with data analysis. Let's check the:

- Number of rows and columns:

In [15]:
data.shape

(83567, 8)

- Top and bottom rows: 

In [16]:
data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN
0           29070  190101010600  270.0   36.0   NaN    18.0  NaN  NaN
1           29070  190101011300  290.0   18.0   NaN    19.0  NaN  NaN
2           29070  190101012000    NaN    0.0   NaN    15.0  NaN  NaN
3           29070  190101020600  180.0   18.0   NaN    21.0  NaN  NaN
4           29070  190101021300  180.0   22.0   NaN    22.0  NaN  NaN

In [17]:
data.tail()

STATION_NUMBER          TIME   DIR  SPEED  GUST  TEMP_F  MAX  MIN
83562           29070  201910011900  47.0   38.0  44.0    51.0  NaN  NaN
83563           29070  201910012000  45.0   39.0  49.0    51.0  NaN  NaN
83564           29070  201910012100  47.0   38.0  49.0    50.0  NaN  NaN
83565           29070  201910012200  46.0   40.0  47.0    49.0  NaN  NaN
83566           29070  201910012300  44.0   41.0  51.0    48.0  NaN  NaN

- Data types of the columns: 

In [18]:
data.dtypes

STATION_NUMBER      int64
TIME                int64
DIR               float64
SPEED             float64
GUST              float64
TEMP_F            float64
MAX               float64
MIN               float64
dtype: object

- Descriptive statistics:

In [19]:
data.describe()

STATION_NUMBER          TIME           DIR         SPEED        GUST  \
count         83567.0  8.356700e+04  80901.000000  82001.000000  729.000000   
mean          29070.0  2.006396e+11    185.428561     15.549359   24.801097   
std               0.0  2.762736e+09     96.434391      8.215270    8.761818   
min           29070.0  1.901010e+11      1.000000      0.000000   11.000000   
25%           29070.0  2.009081e+11    120.000000      9.000000   17.000000   
50%           29070.0  2.014103e+11    200.000000     14.000000   25.000000   
75%           29070.0  2.017043e+11    250.000000     20.000000   31.000000   
max           29070.0  2.019100e+11    360.000000     62.000000   51.000000   

             TEMP_F           MAX           MIN  
count  83555.000000  11136.000000  11136.000000  
mean      40.197690     42.405711     37.884429  
std       14.282431     14.310085     14.789456  
min      -18.000000    -13.000000    -19.000000  
25%       32.000000     33.000000     29.000000  
50%       39.000000     41.000000     37.000000  
75%       51.000000     54.000000     49.000000  
max       81.000000     83.000000     73.000000

Here we can see that there are varying number of observations per column (see the `count` information), because some of the columns have missing values.

## Using your own functions in Pandas 

Now it's again time to convert temperatures from Fahrenheit to Celsius! Yes, we have already done this many times before, but this time we will learn how to apply our own functions to data in a pandas DataFrame.

**We will define a function for the temperature conversion, and apply this function for each Celsius value on each row of the DataFrame. Output celsius values should be stored in a new column called** `TEMP_C`.

We will first see how we can apply the function row-by-row using a `for` loop and then we will learn how to apply the method to all rows more efficiently all at once.

### Defining the function

For both of these approaches, we first need to define our temperature conversion function from Fahrenheit to Celsius:

In [20]:
def fahr_to_celsius(temp_fahrenheit):
    """Function to convert Fahrenheit temperature into Celsius.

    Parameters
    ----------

    temp_fahrenheit: int | float
        Input temperature in Fahrenheit (should be a number)
        
    Returns
    -------
    
    Temperature in Celsius (float)
    """

    # Convert the Fahrenheit into Celsius
    converted_temp = (temp_fahrenheit - 32) / 1.8
    
    return converted_temp

Let's test the function with some known value:

In [21]:
fahr_to_celsius(32)

0.0

Let's also print out the first rows of our data frame to see our input data before further processing: 

In [22]:
data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN
0           29070  190101010600  270.0   36.0   NaN    18.0  NaN  NaN
1           29070  190101011300  290.0   18.0   NaN    19.0  NaN  NaN
2           29070  190101012000    NaN    0.0   NaN    15.0  NaN  NaN
3           29070  190101020600  180.0   18.0   NaN    21.0  NaN  NaN
4           29070  190101021300  180.0   22.0   NaN    22.0  NaN  NaN

### Iterating over rows

We can apply the function one row at a time using a `for` loop and the [iterrows()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html) method. In other words, we can use the `iterrows()` method and a `for` loop to repeat a process *for each row in a Pandas DataFrame* . Please note that iterating over rows is a rather inefficient approach, but it is still useful to understand the logic behind the iteration.

When using the `iterrows()` method it is important to understand that `iterrows()` accesses not only the values of one row, but also the `index` of the row as well. 

Let's start with a simple for loop that goes through each row in our DataFrame:

In [23]:
# Iterate over the rows
for idx, row in data.iterrows():
    
    # Print the index value
    print('Index:', idx)
    
    # Print the row
    print('Temp F:', row['TEMP_F'], "\n")
    
    break

Index: 0
Temp F: 18.0 



<div class="alert alert-info">

**Breaking a loop**

When developing a for loop, you don't always need to go through the entire loop if you just want to test things out. 
The [break](https://www.tutorialspoint.com/python/python_break_statement.htm) statement in Python terminates the current loop whereever it is placed and we used it here just to test check out the values on the first row.
With a large data, you might not want to print out thousands of values to the screen!
</div>

We can see that the `idx` variable indeed contains the index value at position 0 (the first row) and the `row` variable contains all the data from that given row stored as a pandas `Series`.

- Let's now create an empty column `TEMP_C` for the Celsius temperatures and update the values in that column using the `fahr_to_celsius` function we defined earlier:

In [24]:
# Create an empty float column for the output values
data['TEMP_C'] = 0.0

# Iterate over the rows 
for idx, row in data.iterrows():
    
    # Convert the Fahrenheit to Celsius
    celsius = fahr_to_celsius(row['TEMP_F'])
    
    # Update the value of 'Celsius' column with the converted value
    data.at[idx, 'TEMP_C'] = celsius

<div class="alert alert-info">

**Reminder: .at or .loc?**

Here, you could also use `data.loc[idx, new_column] = celsius` to achieve the same result. 
    
If you only need to access a single value in a DataFrame, [DataFrame.at](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.at.html) is faster compared to [DataFrame.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html), which is designed for accessing groups of rows and columns. 
</div>

Finally, let's see how our dataframe looks like now:

In [25]:
data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29070  190101010600  270.0   36.0   NaN    18.0  NaN  NaN   
1           29070  190101011300  290.0   18.0   NaN    19.0  NaN  NaN   
2           29070  190101012000    NaN    0.0   NaN    15.0  NaN  NaN   
3           29070  190101020600  180.0   18.0   NaN    21.0  NaN  NaN   
4           29070  190101021300  180.0   22.0   NaN    22.0  NaN  NaN   

     TEMP_C  
0 -7.777778  
1 -7.222222  
2 -9.444444  
3 -6.111111  
4 -5.555556

### Applying the function

Pandas DataFrames and Series have a dedicated method `.apply()` for applying functions on columns (or rows!). When using `.apply()`, we pass the function name (without parenthesis!) as an argument to the `apply()` method. Let's start by applying the function to the `TEMP_F` column that contains the temperature values in Fahrenheit:

In [26]:
data['TEMP_F'].apply(fahr_to_celsius)

0        -7.777778
1        -7.222222
2        -9.444444
3        -6.111111
4        -5.555556
           ...    
83562    10.555556
83563    10.555556
83564    10.000000
83565     9.444444
83566     8.888889
Name: TEMP_F, Length: 83567, dtype: float64

The results look logical and we can store them permanently into a new column (overwriting the old values): 

In [27]:
data['TEMP_C'] = data['TEMP_F'].apply(fahr_to_celsius)

We can also apply the function on several columns at once. We can re-order the dataframe at the same time in order to see something else than `NaN` from the `MIN` and `MAX` columns

In [28]:
data[['TEMP_F', 'MIN', 'MAX']].apply(fahr_to_celsius)

TEMP_F  MIN  MAX
0      -7.777778  NaN  NaN
1      -7.222222  NaN  NaN
2      -9.444444  NaN  NaN
3      -6.111111  NaN  NaN
4      -5.555556  NaN  NaN
...          ...  ...  ...
83562  10.555556  NaN  NaN
83563  10.555556  NaN  NaN
83564  10.000000  NaN  NaN
83565   9.444444  NaN  NaN
83566   8.888889  NaN  NaN

[83567 rows x 3 columns]

#### Check your understanding

Convert `'TEMP_F'`, `'MIN'`, `'MAX'` to Celsius by applying the function like we did above and store the outputs to  new columns `'TEMP_C'`, `'MIN_C'`, `'MAX_C'`.

In [29]:
# Solution
data[['TEMP_C', 'MIN_C', 'MAX_C']]  = data[['TEMP_F', 'MIN', 'MAX']].apply(fahr_to_celsius)


Applying the function on all columns `data.apply(fahr_to_celsius)` would not give an error in our case, but the results also don't make much sense for columns where input data was other than Fahrenheit temperatures.

You might also notice that our conversion function would also allow us to 
pass one column or the entire dataframe as a parameter. For example, like this: `fahr_to_celsius(data["TEMP_F"])`. However, the code is perhaps easier to follow when using the apply method.

Let's check the output:

In [30]:
data.head(10)

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29070  190101010600  270.0   36.0   NaN    18.0  NaN  NaN   
1           29070  190101011300  290.0   18.0   NaN    19.0  NaN  NaN   
2           29070  190101012000    NaN    0.0   NaN    15.0  NaN  NaN   
3           29070  190101020600  180.0   18.0   NaN    21.0  NaN  NaN   
4           29070  190101021300  180.0   22.0   NaN    22.0  NaN  NaN   
5           29070  190101022000  180.0   22.0   NaN    27.0  NaN  NaN   
6           29070  190101030600  200.0   22.0   NaN    20.0  NaN  NaN   
7           29070  190101031300  230.0   26.0   NaN    26.0  NaN  NaN   
8           29070  190101032000  230.0   26.0   NaN    27.0  NaN  NaN   
9           29070  190101040600    NaN    0.0   NaN    26.0  NaN  NaN   

     TEMP_C  MIN_C  MAX_C  
0 -7.777778    NaN    NaN  
1 -7.222222    NaN    NaN  
2 -9.444444    NaN    NaN  
3 -6.111111    NaN    NaN  
4 -5.555556    NaN    NaN  
5 -2.777778    NaN    NaN  
6 -6.666667    NaN    NaN  
7 -3.333333    NaN    NaN  
8 -2.777778    NaN    NaN  
9 -3.333333    NaN    NaN

<div class="alert alert-info">

**Should I use .iterrows() or .apply()?**

We are teaching the `.iterrows()` method because it helps to understand the structure of a DataFrame and the process of looping through DataFrame rows. However, using `.apply()` is often more efficient in terms of execution time. 

At this point, the most important thing is that you understand what happens when you are modifying the values in a pandas DataFrame. When doing the course exercises, either of these approaches is ok!
</div>

## Parsing dates

We will eventually want to group our data based on month in order to see if April temperatures in 2019 were higher than average. Currently, the date and time information is stored in the column `TIME` (which was originally titled `YR--MODAHRMN`:

`YR--MODAHRMN = YEAR-MONTH-DAY-HOUR-MINUTE IN GREENWICH MEAN TIME (GMT)`

Let's have a closer look at the date and time information we have by checking the values in that column, and their data type:

In [31]:
data['TIME'].head(19)

0     190101010600
1     190101011300
2     190101012000
3     190101020600
4     190101021300
5     190101022000
6     190101030600
7     190101031300
8     190101032000
9     190101040600
10    190101041300
11    190101042000
12    190101050600
13    190101051300
14    190101052000
15    190101060600
16    190101061300
17    190101062000
18    190101070600
Name: TIME, dtype: int64

In [32]:
data['TIME'].tail(10)

83557    201910011400
83558    201910011500
83559    201910011600
83560    201910011700
83561    201910011800
83562    201910011900
83563    201910012000
83564    201910012100
83565    201910012200
83566    201910012300
Name: TIME, dtype: int64

The `TIME` column contains several observations per day (and even several observations per hour). The timestamp for the first observation is `190601010600`, i.e. from 1st of January 1906 (way back!), and the timestamp for the latest observation is `201910012350`.

In [33]:
data['TIME'].dtypes

dtype('int64')

The information is stored as integer values.

We would want to **aggregate the data on a monthly level**, and in order to do so we need to "label" each row of data based on the month when the record was observed. In order to do this, we need to somehow separate information about the year and month for each row.

In practice, we can create a new column (or an index) containing information about the month (including the year, but excluding days, hours and minutes).

Before further processing, we want to convert the `TIME` column as character strings for convenience:

In [34]:
# Convert to string
data['TIME_STR'] = data['TIME'].astype(str)


### String slicing

It is possible to convert the date and time information into character strings and "cut" the needed information from the [string objects](https://docs.python.org/3/tutorial/introduction.html#strings). If we look at the latest time stamp in the data (`201910012350`), you can see that there is a systematic pattern `YEAR-MONTH-DAY-HOUR-MINUTE`. Four first characters represent the year, and six first characters are year + month!

Based on this information, we can slice the correct range of characters from the `TIME_STR` column using [pandas.Series.str.slice()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.slice.html)


In [35]:
# SLice the string
data['YEAR_MONTH'] = data['TIME_STR'].str.slice(start=0, stop=6)

# Let's see what we have
data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29070  190101010600  270.0   36.0   NaN    18.0  NaN  NaN   
1           29070  190101011300  290.0   18.0   NaN    19.0  NaN  NaN   
2           29070  190101012000    NaN    0.0   NaN    15.0  NaN  NaN   
3           29070  190101020600  180.0   18.0   NaN    21.0  NaN  NaN   
4           29070  190101021300  180.0   22.0   NaN    22.0  NaN  NaN   

     TEMP_C  MIN_C  MAX_C      TIME_STR YEAR_MONTH  
0 -7.777778    NaN    NaN  190101010600     190101  
1 -7.222222    NaN    NaN  190101011300     190101  
2 -9.444444    NaN    NaN  190101012000     190101  
3 -6.111111    NaN    NaN  190101020600     190101  
4 -5.555556    NaN    NaN  190101021300     190101

Nice! Now we have "labeled" the rows based on information about day of the year and hour of the day.

### Check your understanding

Create a new column `'MONTH'` with information about the month without the year.

In [36]:
# Extract information about month from the TIME_STR column into a new column 'MONTH':


YEAR_MONTH MONTH
0         190101    01
1         190101    01
2         190101    01
3         190101    01
4         190101    01
...          ...   ...
83562     201910    10
83563     201910    10
83564     201910    10
83565     201910    10
83566     201910    10

[83567 rows x 2 columns]

In [ ]:
# Check the result:


### Datetime (optional for Lesson 6)

In pandas, we can convert dates and times into a new data type [datetime](https://docs.python.org/3.7/library/datetime.html) using [pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) function.

In [37]:
# Convert character strings to datetime
data['DATE'] = pd.to_datetime(data['TIME_STR'])

In [38]:
# Check the output
data['DATE'].head()

0   1901-01-01 06:00:00
1   1901-01-01 13:00:00
2   1901-01-01 20:00:00
3   1901-01-02 06:00:00
4   1901-01-02 13:00:00
Name: DATE, dtype: datetime64[ns]

<div class="alert alert-info">

**Pandas Series datetime properties**

There are several methods available for accessing information about the properties of datetime values. Read more from the pandas documentation about [datetime properties](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetime-properties).
</div>

Now, we can extract different time units based on the datetime-column using the [pandas.Series.dt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) accessor:

In [39]:
data['DATE'].dt.year

0        1901
1        1901
2        1901
3        1901
4        1901
         ... 
83562    2019
83563    2019
83564    2019
83565    2019
83566    2019
Name: DATE, Length: 83567, dtype: int64

In [40]:
data['DATE'].dt.month

0         1
1         1
2         1
3         1
4         1
         ..
83562    10
83563    10
83564    10
83565    10
83566    10
Name: DATE, Length: 83567, dtype: int64

We can also combine the datetime functionalities with other methods from pandas. For example, we can check the number of unique years in our input data: 

In [41]:
data['DATE'].dt.year.nunique()

24

For the final analysis, we need combined information of the year and month. One way to achieve this is to use the  `format` parameter to define the output datetime format according to [strftime(format)](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) method:

In [42]:
# Convert to datetime and keep only year and month
data['YEAR_MONTH_DT'] = pd.to_datetime(data['TIME_STR'], format='%Y%m', exact=False)

`exact=False` finds the characters matching the specified format and drops out the rest (days, hours and minutes are excluded in the output).

In [43]:
data['YEAR_MONTH_DT']

0       1901-01-01
1       1901-01-01
2       1901-01-01
3       1901-01-01
4       1901-01-01
           ...    
83562   2019-10-01
83563   2019-10-01
83564   2019-10-01
83565   2019-10-01
83566   2019-10-01
Name: YEAR_MONTH_DT, Length: 83567, dtype: datetime64[ns]

Now we have a unique label for each month as a datetime object.

## Aggregating data in Pandas by grouping

Here, we will learn how to use [pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) which is a handy method for compressing large amounts of data and computing statistics for subgroups.

We will use the groupby method to calculate the average temperatures for each month trough these main steps:

  1. **grouping the data** based on year and month
  2. Calculating the average for each month (each group) 
  3. Storing those values into **a new DataFrame** `monthly_data`

Before we start grouping the data, let's once more check how our input data looks like:

In [44]:
print("number of rows:", len(data))

number of rows: 83567


In [45]:
data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29070  190101010600  270.0   36.0   NaN    18.0  NaN  NaN   
1           29070  190101011300  290.0   18.0   NaN    19.0  NaN  NaN   
2           29070  190101012000    NaN    0.0   NaN    15.0  NaN  NaN   
3           29070  190101020600  180.0   18.0   NaN    21.0  NaN  NaN   
4           29070  190101021300  180.0   22.0   NaN    22.0  NaN  NaN   

     TEMP_C  MIN_C  MAX_C      TIME_STR YEAR_MONTH MONTH                DATE  \
0 -7.777778    NaN    NaN  190101010600     190101    01 1901-01-01 06:00:00   
1 -7.222222    NaN    NaN  190101011300     190101    01 1901-01-01 13:00:00   
2 -9.444444    NaN    NaN  190101012000     190101    01 1901-01-01 20:00:00   
3 -6.111111    NaN    NaN  190101020600     190101    01 1901-01-02 06:00:00   
4 -5.555556    NaN    NaN  190101021300     190101    01 1901-01-02 13:00:00   

  YEAR_MONTH_DT  
0    1901-01-01  
1    1901-01-01  
2    1901-01-01  
3    1901-01-01  
4    1901-01-01

We have quite a few rows of weather data, and several observations per day. Our goal is to create an aggreated data frame that would have only one row per month.

Let's **group** our data based on unique year and month combinations

In [46]:
grouped = data.groupby('YEAR_MONTH')

<div class="alert alert-info">

**Note**

It would be also possible to create combinations of years and months on-the-fly when grouping the data:
    
```
# Group the data 
grouped = data.groupby(['YEAR', 'MONTH'])
```
</div>

Let's explore the new variable `grouped`:

In [47]:
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [48]:
len(grouped)

256

We have a new object with type `DataFrameGroupBy` with 82 groups. In order to understand what just happened, let's also check the number of unique year and month combinations in our data:

In [49]:
data['YEAR_MONTH'].nunique()

256

Length of the grouped object should be the same as the number of unique values in the column we used for grouping. For each unique value, there is a group of data.

Let's explore our grouped data further. 

Check the "names" of each group

In [51]:
# Next line will print out all 82 group "keys"
grouped.groups.keys()

dict_keys(['190101', '190102', '190103', '190104', '190105', '190106', '190107', '190108', '190109', '190110', '190111', '190112', '190201', '190202', '190203', '190204', '190205', '190206', '190207', '190208', '190209', '190210', '190211', '190212', '190301', '190302', '190303', '190304', '190305', '190306', '190307', '190308', '190309', '190310', '190311', '190312', '190401', '190402', '190403', '190404', '190405', '190406', '190407', '190408', '190409', '190410', '190411', '190412', '190501', '190502', '190503', '190504', '190505', '190506', '190507', '190508', '190509', '190510', '190511', '190512', '200106', '200202', '200207', '200210', '200302', '200306', '200311', '200402', '200403', '200404', '200405', '200406', '200407', '200408', '200409', '200410', '200411', '200412', '200501', '200502', '200503', '200504', '200505', '200506', '200507', '200508', '200509', '200510', '200511', '200512', '200601', '200602', '200603', '200604', '200605', '200606', '200607', '200608', '200609',

**Accessing data for one group:**

- Let's check the contents for a group representing August 2019 (name of that group is `(2019, 4)` if you grouped the data based on datetime columns `YEAR` and `MONTH`). We can get the values of that hour from the grouped object using the `get_group()` method:

In [52]:
# Specify a month (as character string)
month = "190512"

# Select the group
group1 = grouped.get_group(month)

In [53]:
# Let's see what we have
group1

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
5365           29070  190512010600  200.0   36.0   NaN    35.0  NaN  NaN   
5366           29070  190512011300  250.0   45.0   NaN    36.0  NaN  NaN   
5367           29070  190512012000  230.0   45.0   NaN    36.0  NaN  NaN   
5368           29070  190512020600  200.0   62.0   NaN    36.0  NaN  NaN   
5369           29070  190512021300  230.0   62.0   NaN    39.0  NaN  NaN   
...              ...           ...    ...    ...   ...     ...  ...  ...   
5451           29070  190512300600  180.0    5.0   NaN    10.0  NaN  NaN   
5452           29070  190512301300  360.0    9.0   NaN    15.0  NaN  NaN   
5453           29070  190512302000   90.0    5.0   NaN     5.0  NaN  NaN   
5454           29070  190512310600  200.0   18.0   NaN    13.0  NaN  NaN   
5455           29070  190512311300  230.0   18.0   NaN    24.0  NaN  NaN   

         TEMP_C  MIN_C  MAX_C      TIME_STR YEAR_MONTH MONTH  \
5365   1.666667    NaN    NaN  190512010600     190512    12   
5366   2.222222    NaN    NaN  190512011300     190512    12   
5367   2.222222    NaN    NaN  190512012000     190512    12   
5368   2.222222    NaN    NaN  190512020600     190512    12   
5369   3.888889    NaN    NaN  190512021300     190512    12   
...         ...    ...    ...           ...        ...   ...   
5451 -12.222222    NaN    NaN  190512300600     190512    12   
5452  -9.444444    NaN    NaN  190512301300     190512    12   
5453 -15.000000    NaN    NaN  190512302000     190512    12   
5454 -10.555556    NaN    NaN  190512310600     190512    12   
5455  -4.444444    NaN    NaN  190512311300     190512    12   

                    DATE YEAR_MONTH_DT  
5365 1905-12-01 06:00:00    1905-12-01  
5366 1905-12-01 13:00:00    1905-12-01  
5367 1905-12-01 20:00:00    1905-12-01  
5368 1905-12-02 06:00:00    1905-12-01  
5369 1905-12-02 13:00:00    1905-12-01  
...                  ...           ...  
5451 1905-12-30 06:00:00    1905-12-01  
5452 1905-12-30 13:00:00    1905-12-01  
5453 1905-12-30 20:00:00    1905-12-01  
5454 1905-12-31 06:00:00    1905-12-01  
5455 1905-12-31 13:00:00    1905-12-01  

[91 rows x 16 columns]

Ahaa! As we can see, a single group contains a **DataFrame** with values only for that specific month. Let's check the DataType of this group:

In [54]:
type(group1)

pandas.core.frame.DataFrame

So, one group is a pandas DataFrame! This is really useful, because we can now use all the familiar DataFrame methods for calculating statistics etc for this specific group. 
We can, for example, calculate the average values for all variables using the statistical functions that we have seen already (e.g. mean, std, min, max, median, etc.).

We can do that by using the `mean()` function that we already used during Lesson 5. 

- Let's calculate the mean for following attributes all at once:
   - `DIR`, 
   - `SPEED`, 
   - `GUST`, 
   - `TEMP`, 
   - `TEMP_C`
   - `MONTH` 

In [55]:
# Specify the columns that will be part of the calculation
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C']

# Calculate the mean values all at one go
mean_values = group1[mean_cols].mean()

# Let's see what we have
print(mean_values)

DIR       235.494505
SPEED      26.318681
GUST             NaN
TEMP_F     31.000000
TEMP_C     -0.555556
dtype: float64


Here we saw how you can access data from a single group. For getting information about all groups (all months) we can use a `for` loop or methods available in the grouped object.

**For loops and grouped objects:**

When iterating over the groups in our `DataFrameGroupBy`  object it is important to understand that a single group in our `DataFrameGroupBy` actually contains not only the actual values, but also information about the `key` that was used to do the grouping. Hence, when iterating over the data we need to assign the `key` and the values into separate variables.

- Let's see how we can iterate over the groups and print the key and the data from a single group (again using `break` to only see what is happening).

In [57]:
# Iterate over groups
for key,group in grouped:
    # Print key and group
    print("Key:\n", key)
    print("\nFirst rows of data in this group:\n", group.head())
    
    # Stop iteration with break command
    break

Key:
 190101

First rows of data in this group:
    STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29070  190101010600  270.0   36.0   NaN    18.0  NaN  NaN   
1           29070  190101011300  290.0   18.0   NaN    19.0  NaN  NaN   
2           29070  190101012000    NaN    0.0   NaN    15.0  NaN  NaN   
3           29070  190101020600  180.0   18.0   NaN    21.0  NaN  NaN   
4           29070  190101021300  180.0   22.0   NaN    22.0  NaN  NaN   

     TEMP_C  MIN_C  MAX_C      TIME_STR YEAR_MONTH MONTH                DATE  \
0 -7.777778    NaN    NaN  190101010600     190101    01 1901-01-01 06:00:00   
1 -7.222222    NaN    NaN  190101011300     190101    01 1901-01-01 13:00:00   
2 -9.444444    NaN    NaN  190101012000     190101    01 1901-01-01 20:00:00   
3 -6.111111    NaN    NaN  190101020600     190101    01 1901-01-02 06:00:00   
4 -5.555556    NaN    NaN  190101021300     190101    01 1901-01-02 13:00:00   

  YEAR_MONTH_DT  
0    1901-01-

Okey so from here we can see that the `key` contains the name of the group (year, month).

- Let's see how we can create a DataFrame where we calculate the mean values for all those weather attributes that we were interested in. I will repeat slightly the earlier steps so that you can see and better understand what is happening.

In [58]:
# Create an empty DataFrame for the aggregated values
monthly_data = pd.DataFrame()

# The columns that we want to aggregate
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C']

# Iterate over the groups
for key, group in grouped:
    
   # Calculate mean
   mean_values = group[mean_cols].mean()

   # Add the ´key´ (i.e. the date+time information) into the aggregated values
   mean_values['YEAR_MONTH'] = key

   # Append the aggregated values into the DataFrame
   monthly_data = monthly_data.append(mean_values, ignore_index=True)
   

- Let's see what we have now:

In [59]:
print(monthly_data)

            DIR       GUST      SPEED     TEMP_C     TEMP_F YEAR_MONTH
0    213.684211        NaN  18.537634  -2.652330  27.225806     190101
1    228.243243        NaN  20.238095 -11.574074  11.166667     190102
2    225.970149        NaN  15.924731  -6.338112  20.591398     190103
3    200.952381        NaN  13.066667  -0.327160  31.411111     190104
4    252.372881        NaN  11.955556   2.998805  37.397849     190105
..          ...        ...        ...        ...        ...        ...
251  202.282572  24.000000  14.072758  10.098130  50.176634     201906
252  145.070081  12.500000  12.334232  14.453417  58.016151     201907
253  173.752703  14.000000  13.895946  13.277928  55.900270     201908
254  195.995775  17.100000  15.185915  11.435897  52.584615     201909
255   67.166667  41.058824  30.208333  10.925926  51.666667     201910

[256 rows x 6 columns]


Awesome! Now we have aggregated our data and we have a new DataFrame called `monthly_data` where we have mean values for each month in the data set.

**Mean for all groups at once**

We can also achieve the same result by computing the mean of all columns for all groups in the grouped object:

In [60]:
grouped.mean()

STATION_NUMBER          TIME         DIR      SPEED       GUST  \
YEAR_MONTH                                                                   
190101             29070.0  1.901012e+11  213.684211  18.537634        NaN   
190102             29070.0  1.901021e+11  228.243243  20.238095        NaN   
190103             29070.0  1.901032e+11  225.970149  15.924731        NaN   
190104             29070.0  1.901042e+11  200.952381  13.066667        NaN   
190105             29070.0  1.901052e+11  252.372881  11.955556        NaN   
...                    ...           ...         ...        ...        ...   
201906             29070.0  2.019062e+11  202.282572  14.072758  24.000000   
201907             29070.0  2.019072e+11  145.070081  12.334232  12.500000   
201908             29070.0  2.019082e+11  173.752703  13.895946  14.000000   
201909             29070.0  2.019092e+11  195.995775  15.185915  17.100000   
201910             29070.0  2.019100e+11   67.166667  30.208333  41.058824   

               TEMP_F        MAX        MIN     TEMP_C      MIN_C      MAX_C  
YEAR_MONTH                                                                    
190101      27.225806        NaN        NaN  -2.652330        NaN        NaN  
190102      11.166667        NaN        NaN -11.574074        NaN        NaN  
190103      20.591398        NaN        NaN  -6.338112        NaN        NaN  
190104      31.411111        NaN        NaN  -0.327160        NaN        NaN  
190105      37.397849        NaN        NaN   2.998805        NaN        NaN  
...               ...        ...        ...        ...        ...        ...  
201906      50.176634  53.183333  47.850000  10.098130   8.805556  11.768519  
201907      58.016151  60.451613  55.741935  14.453417  13.189964  15.806452  
201908      55.900270  58.048387  53.919355  13.277928  12.177419  14.471326  
201909      52.584615  54.483333  50.466667  11.435897  10.259259  12.490741  
201910      51.666667  52.500000  49.000000  10.925926   9.444444  11.388889  

[256 rows x 11 columns]

## Detecting warm months

Now, we have aggregated our data on monthly level and all we need to do is to check which years had the warmest April temperatures. A simple approach is to select all Aprils from the data, group the data and check which group(s) have the highest mean value:

- select all records that are from April (regardless of the year):

In [61]:
aprils = data[data['MONTH']=="04"]

- take a subset of columns that might contain interesting information:

In [62]:
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C','YEAR_MONTH']]

- group by year and month:

In [63]:
grouped = aprils.groupby(by='YEAR_MONTH')

- calculate mean for each group:

In [64]:
monthly_mean = grouped.mean()

In [65]:
monthly_mean.head()

STATION_NUMBER     TEMP_F    TEMP_C
YEAR_MONTH                                     
190104               29070  31.411111 -0.327160
190204               29070  23.866667 -4.518519
190304               29070  32.764045  0.424469
190404               29070  33.400000  0.777778
190504               29070  27.444444 -2.530864

- check the highest temperature values (sort the data frame in a descending order):

In [66]:
monthly_mean.sort_values(by='TEMP_C', ascending=False).head(10)

STATION_NUMBER     TEMP_F    TEMP_C
YEAR_MONTH                                     
201404               29070  35.437326  1.909626
201504               29070  34.437209  1.354005
200404               29070  34.347032  1.303907
201604               29070  34.303199  1.279555
200804               29070  34.241667  1.245370
201904               29070  33.860918  1.033843
200604               29070  33.836134  1.020075
201104               29070  33.429167  0.793981
190404               29070  33.400000  0.777778
200504               29070  32.916279  0.509044

How did April 2019 rank at the Tampere Pirkkala observation station? 

## Repeating the data analysis with larger dataset


Finally, let's repeat the data analysis steps above for all the available data we have (!!). First, confirm the path to the **folder** where all the input data are located. 
The idea is, that we will repeat the analysis process for each input file using a (rather long) for loop! Here we have all the main analysis steps with some additional output info - all in one long code cell:

In [67]:
# Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, 
                   usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], 
                   na_values=['*', '**', '***', '****', '*****', '******'])

# Rename the columns
new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
data = data.rename(columns=new_names)

#Print info about the current input file:
print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
print("NUMBER OF OBSERVATIONS:", len(data))

# Create column
col_name = 'TEMP_C'
data[col_name] = None

# Convert tempetarues from Fahrenheits to Celsius
data['TEMP_C'] = data['TEMP_F'].apply(fahr_to_celsius)

# Convert TIME to string 
data['TIME_STR'] = data['TIME'].astype(str)

# Parse year and month
data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

# Extract observations for the months of April 
aprils = data[data['MONTH']==4]

# Take a subset of columns
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

# Group by year and month
grouped = aprils.groupby(by=['YEAR', 'MONTH'])

# Get mean values for each group
monthly_mean = grouped.mean()

# Print info
print(monthly_mean.sort_values(by='TEMP_C', ascending=False).head(5))
print("\n")

STATION NUMBER: 29070
NUMBER OF OBSERVATIONS: 83567
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
2014 4               29070  35.437326  1.909626
2015 4               29070  34.437209  1.354005
2004 4               29070  34.347032  1.303907
2016 4               29070  34.303199  1.279555
2008 4               29070  34.241667  1.245370




We will use the `glob()` function from the module `glob` to list our input files. 

In [68]:
import glob

In [69]:
file_list = glob.glob(r'data/0*txt')

<div class="alert alert-info">

**Note**

Note that we're using the \* character as a wildcard, so any file that starts with `data/0` and ends with `txt` will be added to the list of files we will iterate over. We specifically use `data/0` as the starting part of the file names to avoid having our metadata files included in the list!
</div>

In [70]:
print("Number of files in the list", len(file_list))
print(file_list)

Number of files in the list 2
['data/029070.txt', 'data/029500.txt']


Now, you should have all the relevant file names in a list, and we can loop over the list using a for-loop:

In [71]:
for fp in file_list:
    print(fp)

data/029070.txt
data/029500.txt


In [72]:
# Repeat the analysis steps for each input file:
for fp in file_list:

    # Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
    data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

    # Rename the columns
    new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
    data = data.rename(columns=new_names)

    #Print info about the current input file:
    print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
    print("NUMBER OF OBSERVATIONS:", len(data))

    # Create column
    col_name = 'TEMP_C'
    data[col_name] = None

    # Convert tempetarues from Fahrenheits to Celsius
    data['TEMP_C'] = data['TEMP_F'].apply(fahr_to_celsius)

    # Convert TIME to string 
    data['TIME_STR'] = data['TIME'].astype(str)

    # Parse year and month
    data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
    data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

    # Extract observations for the months of April 
    aprils = data[data['MONTH']==4]

    # Take a subset of columns
    aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

    # Group by year and month
    grouped = aprils.groupby(by=['YEAR', 'MONTH'])

    # Get mean values for each group
    monthly_mean = grouped.mean()

    # Print info
    print(monthly_mean.sort_values(by='TEMP_C', ascending=False).head(5))
    print("\n")

STATION NUMBER: 29070
NUMBER OF OBSERVATIONS: 83567
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
2014 4               29070  35.437326  1.909626
2015 4               29070  34.437209  1.354005
2004 4               29070  34.347032  1.303907
2016 4               29070  34.303199  1.279555
2008 4               29070  34.241667  1.245370


STATION NUMBER: 29500
NUMBER OF OBSERVATIONS: 103105
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
2019 4               29500  41.639777  5.355432
2008 4               29500  40.838936  4.910520
2014 4               29500  40.226415  4.570231
2016 4               29500  39.176634  3.987019
2011 4               29500  38.647826  3.693237




How about now, how did April 2019 rank across different stations?